In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [6]:
df = pd.read_csv('NCAA.csv')
df['result'] = 1

In [5]:
def fun(item):
    if item < 556:
        return 0
    else:
        return 1
for group,frame in df.groupby(fun):
    if group == 1:
        df_lose = pd.DataFrame(frame)
    if group == 0:
        df_win = pd.DataFrame(frame)


In [7]:
new_df_lose = pd.read_csv('ncaa_lose.csv').set_index('Unnamed: 0')
for i in np.arange(556,1112):
    gameid = new_df_lose.loc[i,'game_id'].split("-")
    gameid_n = gameid[0]+'-'+gameid[2]+'-'+gameid[1]
    new_df_lose.loc[i,'game_id'] = gameid_n
df = pd.concat([df_win,new_df_lose],axis= 0, sort=False)
df.head()

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,...,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id,result
0,1373,81,1108,77,N,0,W16b,W16a,16,16,...,96.3688,99.6885,76.0664,73.8504,103.4213,99.8665,99.6370,106.4500,2002-1373-1108,1
1,1104,86,1194,78,N,0,Y02,Y15,2,15,...,95.2313,93.8770,71.2357,71.2446,100.2897,96.8669,98.4183,99.9263,2002-1104-1194,1
2,1112,86,1364,81,N,0,Z03,Z14,3,14,...,104.0411,96.9262,63.2345,64.7948,105.2163,105.4534,96.0965,97.6704,2002-1112-1364,1
3,1181,84,1457,37,N,0,Y01,Y16,1,16,...,90.2544,87.7504,68.2128,67.4639,100.2384,95.3632,94.6587,99.9754,2002-1181-1457,1
4,1231,75,1428,56,N,0,Y05,Y12,5,12,...,94.4987,89.9224,62.6181,63.3547,107.7123,108.6231,97.4318,95.6459,2002-1231-1428,1


In [8]:
team_list = list()
for i in df.team1_id:
    if i in team_list:
        continue
    else:
        team_list.append(i)
for i in df.team2_id:
    if i in team_list:
        continue
    else:
        team_list.append(i)


In [9]:
teamname_list = list()
for i in df.team1_teamname:
    if i in teamname_list:
        continue
    else:
        teamname_list.append(i)
for i in df.team2_teamname:
    if i in teamname_list:
        continue
    else:
        teamname_list.append(i)
#print (teamname_list)
print (len(teamname_list))

249


In [10]:
team_id = dict()
for i,o in zip(df.team1_id,df.team1_teamname):
    team_id[i] = o
for i,o in zip(df.team2_id,df.team2_teamname):
    team_id[i] = o
team_ID = pd.Series(data = team_id)
team_ID = pd.DataFrame({'id':team_ID.index, 'teamname':team_ID.values})
team_ID.head()

,id,teamname
0,1373,Siena
1,1104,Alabama
2,1112,Arizona
3,1181,Duke
4,1231,Indiana


In [11]:
result=[]
for i in range(len(df)):
    if df.iloc[i]['team1_score'] > df.iloc[i]['team2_score']:
        result.append(1)
    elif df.iloc[i]['team1_score'] < df.iloc[i]['team2_score']:
        result.append(0)
    else:
        result.append(2)

In [12]:
gamesnum = dict()
for i in teamname_list:
    num = df[df.team1_teamname == i].count()[1]
    gamesnum[i] = num
for i in teamname_list:
    num = df[df.team2_teamname == i].count()[1]
    gamesnum[i] += num

In [13]:
#calculate avg feature
def avg_feature(feature):
    df = pd.read_csv('NCAA.csv')
    feature1 = dict()
    feature2 = dict()
    for i in teamname_list:
        total = df[df.team1_teamname == i]['team1_'+feature].sum()    
        feature1[i] = total
    for i in teamname_list:
        total = df[df.team2_teamname == i]['team2_'+feature].sum()    
        feature1[i] += total
    for x,y in feature1.items():
        feature2[x] = y/gamesnum[x]
    return feature2

In [15]:
#calculate average score per team
avg_score = avg_feature('score')
#avg 2 point
avg_2pt = avg_feature('fg2pct')
#avg 3 point rate
avg_3pt = avg_feature('fg3pct')
#Shooting percentage on free throws
avg_ft = avg_feature('ftpct')
#Points scored per 100 offensive possessions.
avg_oe =avg_feature('oe')
#Percentage of field goals that were preceded by an assist.
avg_arate = avg_feature('arate')
#Blocked shots divided by opponents 2 point field goal attempts.
avg_blk = avg_feature('blockpct')
#Steals divided by defensive possessions
avg_stl = avg_feature('stlrate')
#Opponent’s shooting percentage on 2 point field goals.
opp_2pt = avg_feature('oppfg2pct')
#Opponent’s shooting percentage on 3 point field goals. 
opp_3pt = avg_feature('oppfg3pct')

In [16]:
#create team information dataframe
team_info = pd.DataFrame.from_dict(avg_score, orient = 'index', columns = ["avg_score"])
team_info.reset_index(level=0, inplace=True)
team_info.columns = ['teamname','avg_score']


In [17]:
#map features into dataframe
team_info['avg_3pt'] = team_info['teamname'].map(avg_3pt)
team_info['avg_2pt'] = team_info['teamname'].map(avg_2pt)
team_info['avg_ft'] = team_info['teamname'].map(avg_ft)
team_info['avg_blk'] = team_info['teamname'].map(avg_blk)
team_info['avg_stl'] = team_info['teamname'].map(avg_stl)
team_info['avg_assrate'] = team_info['teamname'].map(avg_arate)
team_info['avg_oe'] = team_info['teamname'].map(avg_oe)
team_info['opp_2pt'] = team_info['teamname'].map(opp_2pt)
team_info['opp_3pt'] = team_info['teamname'].map(opp_3pt)


In [18]:
team_info_n = pd.merge(team_info,team_ID)
team_info = team_info_n.sort_values(by = 'teamname')
team_info = team_info.reset_index().drop('index',axis = 1)
team_info.head()

,teamname,avg_score,avg_3pt,avg_2pt,avg_ft,avg_blk,avg_stl,avg_assrate,avg_oe,opp_2pt,opp_3pt,id
0,Air Force,60.500000,38.995300,57.903900,73.003700,7.591700,0.142800,65.432400,111.359150,49.899600,30.836200,1102
1,Akron,54.000000,34.008833,48.686233,68.695567,9.945933,0.103500,56.494600,103.632200,44.991133,31.390967,1103
2,Alabama,70.384615,34.348588,50.447966,70.849239,10.984112,0.096557,49.686451,107.457154,45.399986,33.907577,1104
3,Alabama A&M,69.000000,30.347300,44.644500,61.384600,9.137900,0.150100,56.671900,92.704000,47.413800,30.434800,1105
4,Alabama St,55.000000,31.690133,47.678633,63.238967,9.891233,0.101033,52.966300,97.447433,43.448533,33.516867,1106


In [19]:
def caldif(t1, t2):
    team_1 = team_info[team_info.teamname == t1].drop(columns=['teamname','id']).values[0]
    team_2 = team_info[team_info.teamname == t2].drop(columns=['teamname','id']).values[0]
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [23]:
X =list()
for x,y in zip(df['team1_teamname'],df['team2_teamname']):
    X.append(caldif(x,y))

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [25]:
y = df['result']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 0)

In [31]:
X_train_scale = scaler.fit_transform(X_train)
X_test_scale =scaler.fit_transform(X_test)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
lr = LogisticRegression()
lr.fit(X_train_scale, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
predict = lr.predict(X_test)

In [36]:
lr.predict_proba(X_test_scale)[:15]

array([[0.70717709, 0.29282291],
       [0.63197635, 0.36802365],
       [0.63772909, 0.36227091],
       [0.20929902, 0.79070098],
       [0.85671529, 0.14328471],
       [0.47418134, 0.52581866],
       [0.53107709, 0.46892291],
       [0.87923409, 0.12076591],
       [0.90095798, 0.09904202],
       [0.5124141 , 0.4875859 ],
       [0.37228919, 0.62771081],
       [0.45950861, 0.54049139],
       [0.8937098 , 0.1062902 ],
       [0.65738351, 0.34261649],
       [0.37398236, 0.62601764]])

In [25]:
lr.score(X_train_scale, y_train)

0.6786570743405276

In [37]:
#random forest
clf = RandomForestClassifier(n_estimators=23,max_depth = 6,random_state=0).fit(X_train, y_train)
print (clf.score(X_train, y_train))
print (clf.score(X_test, y_test))

0.8405275779376499
0.7338129496402878


In [38]:
#gradient boosted
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 0, learning_rate= 0.01).fit(X_train,y_train)
print (gb.score(X_train, y_train))
print (gb.score(X_test, y_test))

0.7446043165467626
0.7050359712230215


In [39]:
predict_2018 = pd.read_csv('example.csv')

In [40]:
predict_2018.head()

,id,probability
0,2017-1274-1277,0.457031
1,2017-1242-1277,0.962555
2,2017-1124-1308,0.993971
3,2017-1374-1425,0.420066
4,2017-1242-1413,0.997231


In [41]:
def caldifid(t1, t2):
    team_1 = team_info[team_info.id == t1].drop(columns=['teamname','id']).values[0]
    team_2 = team_info[team_info.id == t2].drop(columns=['teamname','id']).values[0]
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [42]:
X_predict = list()
for i in np.arange(len(predict_2018)):
    matchid = predict_2018.loc[i,'id'].split('-')
    team1 = int(matchid[1])
    team2 = int(matchid[2])
    n = caldifid(team1,team2)
    X_predict.append(n)

In [43]:
X_predict_scale = scaler.fit_transform(X_predict)
prob = clf.predict_proba(X_predict)
prob[:15]

array([[0.43320866, 0.56679134],
       [0.2949711 , 0.7050289 ],
       [0.25537604, 0.74462396],
       [0.62916711, 0.37083289],
       [0.14192867, 0.85807133],
       [0.26836986, 0.73163014],
       [0.36802574, 0.63197426],
       [0.29975596, 0.70024404],
       [0.18662269, 0.81337731],
       [0.29349606, 0.70650394],
       [0.36677291, 0.63322709],
       [0.18225998, 0.81774002],
       [0.41500959, 0.58499041],
       [0.42713748, 0.57286252],
       [0.55477324, 0.44522676]])

In [44]:
for i in np.arange(len(predict_2018)):
    predict_2018.iloc[i,1]=prob[i][0]

In [45]:
predict_2018.to_csv('Initial_Prediction.csv')